In [ ]:
import cv2
from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import backend as K
tf.compat.v1.disable_eager_execution() #to use K.gradients!

K.clear_session()

In [ ]:
def show_image(img, is_bgr=True):
    """Function to display image
        Args: 
            img: numpy.ndarray
            is_bgr: bool. True: accepts BGR image.

        Returns: 

    """
    if is_bgr:
        display(Image.fromarray(img[:,:,::-1]))
    else:
        display(Image.fromarray(img))

In [ ]:
#TODO: update this function for your preprocessing steps!
def preprocess_image(img_path: str, img_size=img_size):
    """Function to prepare image for prediction
        Args: 
            img_path: str
            img_size: int. 224 for vgg16, 299 for xception
            
        Returns:
            numpy.ndarray. (1, img_size, img_size, 3)

    """
    #load image from file
    x = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_size, img_size))
    
    #preprocessing steps
    x = tf.keras.preprocessing.image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.xception.preprocess_input(x) #tl - xception
    return x

### prepare model

In [ ]:
#load model from file
model = tf.keras.models.load_model("<model_path>")

### ! TODO: update image path !

In [ ]:
img_path = '<test_image_path>'
img_size = <image_size>

In [ ]:
#show image
original_image = cv2.imread(img_path)
show_image(original_image)

### predict (single image)

In [ ]:
### predict (single image)#preprocess image
img_ = preprocess_image(img_path)

#predict
preds = model.predict(img_)


In [ ]:
#print layer names and select the conv layer!
for layer in model.layers:
    if layer.name == 'xception':
        for layer2 in layer.layers:
            print('*',layer2.name)
    else:
        print(layer.name)

In [ ]:
#TODO: update following lines for your model

#!we need to get output for xception model first! 
xception_output = model.get_layer('xception').output[:, np.argmax(preds[0])]

last_conv_layer = model.get_layer('xception').get_layer("block14_sepconv2_act") #TODO: update layer name

In [ ]:
#check layer output
print(last_conv_layer.output)
last_cn_size=last_conv_layer.output.shape[-1] #2048

#### get gradients

In [ ]:
#inspired by https://github.com/nickbiso/Keras-Class-Activation-Map
grads = K.gradients(xception_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads, axis=(0, 1, 2))

iterate = K.function([ model.get_layer('xception').input], [pooled_grads, last_conv_layer.output[0]])

pooled_grads_value, conv_layer_output_value = iterate([img_])

for i in range(last_cn_size):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

### visualize heatmap

In [ ]:
plt.matshow(heatmap)
plt.show()


In [ ]:

#copy original image
img = original_image.copy()

#resize heatmap 
heatmap_ = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

#update pixel values : 0-1 > 0-255
heatmap_ = np.uint8(255 * heatmap_)

#combine heatmap with the original image.
heatmap_ = cv2.applyColorMap(heatmap_, cv2.COLORMAP_JET)

result_img = cv2.addWeighted(heatmap_, 0.2, img, 0.8, 0)
result_img = np.uint8(result_img)

show_image(result_img)
print(preds)
